# 11장 자연어처리 3부

**감사말**: 프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 10장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

**tensorflow 버전과 GPU 확인**
- 구글 코랩 설정: '런타임 -> 런타임 유형 변경' 메뉴에서 GPU 지정 후 아래 명령어 실행 결과 확인

    ```
    !nvidia-smi
    ```

- 사용되는 tensorflow 버전 확인

    ```python
    import tensorflow as tf
    tf.__version__
    ```
- tensorflow가 GPU를 사용하는지 여부 확인

    ```python
    tf.config.list_physical_devices('GPU')
    ```

## 11.4 트랜스포머 아키텍처

2017년 논문 
["Attention is all you need"](https://arxiv.org/abs/1706.03762)에서
소개된 트랜스포머(Transformer) 아키텍처는 자연어처리 분야에서 혁명을 불러왔다.
트랜스포머는 "**뉴럴 어텐션**(Neural Attention)" 기법을 이용하여 
순환층 또는 합성곱 층과는 다르게 작동하는 순차 모델(sequence model)을 구현한다.

여기서는 뉴럴 어텐션의 작동법을 설명한 후에
트랜스포머 인코더를 이용하여 IMDB 영화 리뷰 모델을 구현한다. 

### 11.4.1 셀프 어텐션(self-attention)

입력값의 특성 중에 보다 중요한 특성에 **집중(attention)**하면 보다 효율적으로
훈련이 진행될 수 있다. 
아래 그림이 집중(attention)이 어떻게 활용되는지 잘 보여준다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-05.png" style="width:60%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

앞서 유사한 아이디어를 활용한 적이 있다.

- 합성곱 신경망의 맥스 풀링(max pooling): 지역적으로 가장 중요한 특성만 사용한다.
- TF-IDF 정규화: 텍스트 벡터화를 위해 사용되는 TF-IDF 정규화는
    사용되는 토큰(tokens)에 포함된 정보의 중요도를 평가하여
    보다 중요한 정보를 담은 토큰에 집중한다. 

**셀프 어텐션**(self-attention)은 주어진 문장에 사용된 단어들의 연관성을 평가하여
그 결과를 해당 문장에 적용하여 입력값을 변환하는 기법을 
가리킨다. 
즉, **문맥**(context)를 활용한다.
아래 그림은 "The train  left the station on time." 이라는 문장에
셀프 어텐션을 적용하여 입력값을 변환하는 과정을 나타낸다.

- 1단계: 문장에 사용된 각 토큰들 사이의 연관성 계산.
- 2단계: 계산된 연관성을 (토큰) 벡터와 결합시킨 후 새로운
    토큰 벡터들의 시퀀스 생성.
    아래 그림에서는 "station" 단어에 해당하는 벡터가 변환되는 과정을 보여줌.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-06.png" style="width:70%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

**질문-키-값(query-key-value) 모델**

셀프 어텐션의 작동 과정을 식으로 표현하면 다음과 같다. 

    outputs = sum(inputs * pairwise_scores(inputs, inputs))
                    |                        |      |
                   (C)                      (A)    (B)

위 식은 원래 검색 엔진 또는 추천 시스템에 사용되는 
보다 일반화된 셀프 어텐션의 작동과정을 표현한 식의 특별한 경우를 보여준다. 

    outputs = sum(values * pairwise_scores(query, keys))
                    |                        |      |
                   (C)                      (A)    (B)

예를 들어, 아래 그림은 "dogs on the beach." 질문(query)에 가장 
적절한 사신을 검색한다면 각 사진과의 연관성(keys) 점수를
해당 사진(values)과 결합하여 가장 높은 점수를 갖는 사진을
추천하는 것을 보여준다. 

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-07.png" style="width:60%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

질문-키-값(query-key-value) 모델을 실전에 적용할 때 많은 경우 키(keys)와 값(values)가 동일하다. 

- 기계 번역:
    "How's the weather today?"를 스페인어로 기계 번역하려 할 경우
    스페인어로 날씨에 해당하는 "tiempo"를 키(key)로 해서 주어진 영어 문장(query)에 
    사용된 단어들과 비교해야 한다. 
- 텍스트 분류:
    앞서 셀프 어텐션 설명을 위해 사용된 그림에서처럼 query, keys, values 모두 동일하다.

### 11.4.2 멀티헤드 어텐션

단어들 사이의 연관성을 다양한 방식으로 알아내기 위해 셀프 어텐션을 수행하는 
**헤드**(head)를 여러 개 병렬로 처리한 후에 다시 합치는 기법을
**멀티헤드 어텐션**(multi-head attention)이다.
아래 그림은 두 개의 헤드를 사용하는 것을 보여주며 각각의 헤드가 하는 일은 다음과 같다.

- 질문, 키, 값을 각각 서로 다른 밀집 밀집 층으로 구성된 블록을 통과 시킨다.
- 이후 변환된 질문, 키, 값에 셀프 어텐션을 적용한다.

각 헤드에 포함된 밀집 층 블록으로 인해 멀티헤드 어텐션 층에서도
학습이 이루어진다.

**참고**: 채널 분리 합성곱 층의 알고리즘과 기본 아이디어가 유사하다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-08.png" style="width:70%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

### 11.4.2 트랜스포머 인코더

멀티헤드 어텐션을 밀집(dense) 층, 정규화 층, 잔차 연결 등과 조합하여
**트랜스포머 인코더**(transformer encoder)를 생성한다.

아래 그림에서 사용되는 정규화 층인 케라스의 `LayerNormalization`은
정규화를 배치 단위가 아닌 시퀀스 단위로 정규화를 실행한다.
시퀀스 데이터를 처리할 때는 `BatchNormalization` 보다 잘 작동한다. 

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-09.png" style="width:35%;"></div>

그림 출처: [Deep Learning with Python(Manning MEAP)](https://www.manning.com/books/deep-learning-with-python-second-edition)

**예제: IMDB 데이터셋**

데이터셋을 준비하는 과정은 이전과 동일하다.

In [0]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

if 'google.colab' in str(get_ipython()):
    !rm -r aclImdb/train/unsup
else: 
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

In [0]:
import os, pathlib, shutil, random
from tensorflow import keras

batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
    )
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
    )
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )
text_only_train_ds = train_ds.map(lambda x, y: x)

텍스트 벡터화는 정수들의 벡터를 사용하며, 리뷰의 최대 길이를 600 단어로 제한한다. 

- `output_mode="int"`
- `output_sequence_length=600`

In [0]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

# 어휘 색인화
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

**트랜스포머 구현**

위 그림에서 설명된 트랜스포머 인코더를 층(layer)으로 구현하면 다음과 같다.
생성자의 입력값을 예를 들어 설명하면 다음과 같다.

- `embed_dim`
    - 트랜스포머 인코더는 "단어 임베딩" 층을 통과한 값을 받음.
    - 예를 들어 `embed_dim=256`은 단어 임베딩이
        `(600, 256)` 모양의 샘플을 생성할 것을 기대함.
- `dense_dim`: 밀집 층에서 사용되는 유닛(unit) 수
- `num_heads`: 헤드(head) 수

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더 활용 모델**

훈련 데이터셋이 입력되면 먼저 단어 임베딩을 이용하여 
단어들 사이의 연관성을 찾는다.
이후 트랜스포머 인코더로 셀프 어텐션을 적용한다.

사용되는 변수들은 다음과 같다.

- `vocab_size = 20000`: 어휘 색인 크기
- `embed_dim = 256`: 단어 임베딩 특성 수
- `dense_dim = 32`: 트랜스포머 인코더에 사용되는 밀집층의 유닛(unit) 수
- `num_heads = 2`: 트랜스포머 인코더에 사용되는 밀집층의 헤드(head) 수

In [0]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

훈련 과정은 특별한 게 없다.
테스트셋에 대한 정확도가 87.5% 정도로 바이그램 모델보다 좀 더 낮다.

In [0]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

**모델 비교**

자연어처리와 관련된 모델을 단어 순서 인식과 문맥 이해 차원에서 비교하면 다음과 같다.

| | 단어순서 인식 | 문맥 이해 |
| :---: | :---: | :---: |
| 유니그램 주머니 모델 | X | X |
| 바이그램 주머니 모델 | $\triangle$ | X |
| RNN | O | X |
| 셀프 어텐션 | X | O |
| 트랜스포머 | O | O |

**단어 위치 인코딩**

앞서 살펴 본 트랜스포머 인코더는 셀프 어텐션과 밀집층을 사용하기에
단어순서를 제대로 활용하지는 못한다.
하지만 단어 인코딩 과정에서 단어순서 정보를 활용하도록 하는 기능을 추가하면
트랜스포머가 알아서 단어위치 정보를 활용한다.

다음 `PositionalEmbedding` 층 클래스는 두 개의 임베딩 클래스를 사용한다.
하나는 보통의 단어 임베딩이며,
다른 하나는 단어의 위치 정보를 임베딩한다. 
각 임베딩의 출력값을 합친 값을 트랜스포머에게 전달하는 역할을 수행한다.

In [0]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**단어위치인식 트랜스포머 아키텍처**

아래 코드는 `PositionalEmbedding` 층을 활용하여 트랜스포머 인코더가 
단어위치를 활용할 수 있도록 한다. 
최종 모델의 테스트셋에 대한 정확도가 88.3%까지 향상됨을 확인할 수 있다.

In [0]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

### 11.4.4 트랜스포머 사용 기준

단어주머니 모델이 여전히 유용하게 활용된다.
실제로 IMDB 데이터셋에 대한 성능도 단어주머니 모델이 가장 좋았다.
그리고 많은 실험 결과 "훈련셋의 크기"와 "텍스트의 평균 단어 수"의 비율이 모델 선택에
결정적인 역할을 수행한다는 경험 법칙이 알려졌다.

- ("훈련셋의 크기" $/$ "텍스트의 평균 단어 수") $>$ 1500 인 경우: 트랜스포머 등 순차 모델
- ("훈련셋의 크기" $/$ "텍스트의 평균 단어 수") $<$ 1500 인 경우: 바이그램 단어주머니 모델

**예제 1**

1천개의 단어를 포함한 텍스트 십만 개로 이루어진 훈련셋을 사용하는 경우 비율이 100이기에 바이그램 모델을 사용하는 것이 좋다.

**예제 2**

평균 40개의 단어를 포함하는 트윗(tweets) 5만 개로 이루어진 훈련셋을 사용하는 경우 비율이 1,250이기에 역시 바이그램 모델을 사용하는 것이 좋다.

**예제 3**

평균 40개의 단어를 포함하는 트윗(tweets) 50만 개로 이루어진 훈련셋을 사용하는 경우 비율이 12,500이기에 이번엔 트랜스포머 인코더를 활용하는 것이 좋다.

**예제 4**

IMDB 훈련셋은 2만 개의 리뷰로 구성되며 리뷰 한 개는 평균 233 개의 단어를 포함한다. 비율이 85.84 정도이기에 바이그램 모델이 보다 적합해야 하는데
지금까지 살펴본 결과가 이에 입증한다.

**경험 법칙의 직관적 이해**

짧은 문장이 많을 수록 문맥을 파악하려면 단어들의 순서가 중요하며,
문장에 사용된 단어들 사이의 복잡한 연관성을 보다 주의깊게 살펴볼 필요가 있다.
예를 들어, "그 영화는 실패야"와 "그 영화는 실패였어"는 분명 다른 의미를 가지지만
단어주머니 모델은 차이점을 파악하기 어렵다. 
반면에 보다 긴 문장의 주제와 긍정/부정 등의 감성에 대한 분류는
단어 관련 통계의 중요성이 보다 크다.

**주의사항** 

앞서 설명한 경험 법칙은은 텍스트 분류(text classification)에 한정된다. 
예를 들어 기계 번역(machine translation)의 경우 매우 긴 문장을 다룰 때 
트랜스포머가 가본적으로 가장 강력한 성능의 모델을 생성한다. 